In [1]:
!pip install clang

You should consider upgrading via the 'C:\Users\natal\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [87]:
import sys
import os

print("sys.path:", sys.path) # where is Python is looking for modules?
print("PYTHONPATH:", os.environ.get('PYTHONPATH'))

import clang.cindex
clang.cindex.Config.set_library_file('C:\\Program Files\\LLVM\\bin\\libclang.dll')

# A simple Node class to represent variables in the tree
class VariableNode:
    def __init__(self, name, var_type, scope, line):
        self.name = name
        self.type = var_type
        self.scope = scope
        self.line = line
        self.children = []

    def __repr__(self):
        return f"Variable(name={self.name}, type={self.type}, scope={self.scope}, line={self.line})"

def create_translation_unit(source_file_path, header_file_path=None):
    index = clang.cindex.Index.create()
    
    # Create translation unit with the source and header file
    if header_file_path:
        translation_unit = index.parse(source_file_path, args=['-include', header_file_path])
    else:
        translation_unit = index.parse(source_file_path)
    return translation_unit

# otherwise getNeighborInfo (template __device__ func) body is treated as a comment and ignored by parser
def copy_source_cuda_file_without_template_funcs(source_file_path, copy_file_path):
    with open(source_file_path, 'r') as file:
        code = file.read()

    code = code.replace('\ntemplate', '\n//template')

    with open(copy_file_path, 'w') as file:
        file.write(code)

sys.path: ['c:\\Users\\natal\\Documents\\amr-gpu', 'C:\\Users\\natal\\Downloads\\clang-18.1.8.src\\clang-18.1.8.src\\bindings\\python', 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\\python310.zip', 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\\DLLs', 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\\lib', 'C:\\Users\\natal\\AppData\\Local\\Microsoft\\WindowsApps\\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0', '', 'C:\\Users\\natal\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python310\\site-packages', 'C:\\Users\\natal\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python310\\site-packages\\win32', 'C:\\Users\\natal\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\\Local

In [113]:
source_file_path = "C:\\Users\\natal\\Documents\\amr-gpu\\amr-gpu.cu"
copy_file_path = "C:\\Users\\natal\\Documents\\amr-gpu\\amr-gpu_for-linter.cu"

copy_source_cuda_file_without_template_funcs(source_file_path, copy_file_path)
source_file_path = copy_file_path
#header_file_path = "C:\\Users\\natal\\Documents\\amr-gpu\\amr-gpu.h"

# Create the translation unit including the header file
translation_unit = create_translation_unit(source_file_path) #, header_file_path)

def is_gpu_function(cursor):
    i = 0
    gpu_kw_spelling = None
    for token in cursor.get_tokens():
        if token.spelling in ("__global__", "__device__"):
            gpu_kw_spelling = token.spelling
            break
        i += 1
        if i == 20:
            break

    '''if gpu_kw_spelling is None:   
        i = 0
        for token in cursor.get_children():
            if token.spelling in ("__global__", "__device__"):
                gpu_kw_spelling = token.spelling
                break
            i += 1
            if i == 20:
                break'''

    if gpu_kw_spelling is None:
        return False, None
    return True, gpu_kw_spelling

parent = {None: None}
def traverse_ast(node, scope=('global', None, None), variable_tree=[]):
    curr_scope, gpu_kw, gpu_loop_kw = scope
    new_scope = curr_scope

    # Process nodes only from amr-gpu.cu
    node_location_filepath = str(node.location.file) if node.location.file else "None"
    if node_location_filepath:
        node_location_filepath = os.path.normpath(node_location_filepath)

    if not (node_location_filepath == "None" or node_location_filepath == source_file_path):
        return
    
    node_info = (node.spelling, node.kind, node.location.line, node.location.column)
    if node_info not in parent:
        parent[node_info] = None

    if gpu_kw == '__global__' and node.kind in (clang.cindex.CursorKind.WHILE_STMT, clang.cindex.CursorKind.FOR_STMT):
        gpu_loop_kw = 'while' if node.kind == clang.cindex.CursorKind.WHILE_STMT else 'for'
    
    elif node.kind == clang.cindex.CursorKind.IF_STMT and gpu_kw is not None and (gpu_kw == '__device__' or gpu_loop_kw is not None):
        if gpu_loop_kw is not None:
            print(f'"if" in a "{gpu_loop_kw}" loop at', node.location.line, 'in', gpu_kw, curr_scope)
        else:
            print(f'"if" at', node.location.line, 'in', gpu_kw, curr_scope)

    do_children = True
    if node.kind == clang.cindex.CursorKind.VAR_DECL:
        line = -1
        if node.location.file:
            line = node.location.line
        variable = VariableNode(node.spelling, node.type.spelling, curr_scope, line)
        variable_tree.append(variable)

    elif node.kind in (clang.cindex.CursorKind.FUNCTION_DECL, clang.cindex.CursorKind.FUNCTION_TEMPLATE):
        new_scope = node.spelling

        is_gpu_f = False
        for prev_node in variable_tree[-5:]:
            if type(prev_node) is VariableNode and prev_node.name in ('__global__', '__device__'):
                is_gpu_f = True
                gpu_kw = prev_node.name
        if not is_gpu_f:
            is_gpu_f, gpu_kw = is_gpu_function(node)
        # getIndexInv, getIndex, rhoFunc, getNeighborIdx, checkIfBorder, keyExists, getNeighborInfo, calcGradCell, setGridCell
        # calcGrad, make1lvlGrid, insert_vector_pointers, insert, printHashtable 
        if is_gpu_f:
            variable_tree.append(f"\nEntering GPU {gpu_kw} function: |{new_scope}|")
        else:
            do_children = False
            gpu_kw = None
            #variable_tree.append(f"\nPassing non-gpu function: |{new_scope}|")
    
    # Process all children
    if do_children:
        for child in node.get_children():
            parent[(child.spelling, node.kind, child.location.line, child.location.column)] = node_info
            traverse_ast(child, (new_scope, gpu_kw, gpu_loop_kw), variable_tree=variable_tree)

    return variable_tree

ast = traverse_ast(translation_unit.cursor)

print()
# Output the variable tree with their scopes
for entry in ast:
    print(entry)

"if" at 223 in __device__ getNeighborInfo
"if" in a "while" loop at 336 in __global__ make1lvlGrid
"if" at 353 in __device__ setGridCell
"if" at 362 in __device__ setGridCell
"if" at 365 in __device__ setGridCell
"if" in a "while" loop at 443 in __global__ insert_vector_pointers
"if" in a "while" loop at 470 in __global__ insert
"if" in a "for" loop at 494 in __global__ printHashtable

Variable(name=__device__, type=int, scope=global, line=100)

Entering GPU __device__ function: |getIndexInv|
Variable(name=dim, type=int, scope=getIndexInv, line=101)
Variable(name=scale, type=long, scope=getIndexInv, line=102)
Variable(name=__device__, type=int, scope=global, line=109)

Entering GPU __device__ function: |getIndex|
Variable(name=scale, type=long, scope=getIndex, line=110)
Variable(name=dim, type=int, scope=getIndex, line=112)

Entering GPU __device__ function: |rhoFunc|
Variable(name=rsq, type=double, scope=rhoFunc, line=165)
Variable(name=i, type=short, scope=rhoFunc, line=166)
Variable